In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from pmdarima import auto_arima,arima
import warnings
from statsmodels.api import sm
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [12]:
ari = pd.read_csv("data_ari.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
ari = ari.drop(columns=['Unnamed: 0']).reset_index(drop=True)


In [13]:
ili = pd.read_csv("data_ili.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
ili = ili.drop(columns=['Unnamed: 0']).reset_index(drop=True)

In [ ]:
def eval_metrics(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    if mask.sum() == 0:
        return np.nan, np.nan

    mae = mean_absolute_error(y_true[mask], y_pred[mask])
    rmse = root_mean_squared_error(y_true[mask], y_pred[mask])
    # mae = np.mean(np.abs(y_true[mask] - y_pred[mask]))
    return mae, rmse

In [15]:
def train_data(df,location,date):
    df = df[df['location']==location]
    df = df.set_index('truth_date')
    train = df[df.index<=date]
    test = df[df.index>date]
    return train,test

In [ ]:
def plot_train_test(train,test,model_name,location,aux):
    plt.figure(figsize=(15,17))
    plt.plot(train.index, train["value"], color='blue', label='Training Data')  
    plt.plot(test.index, test["value"], color='orange', label='Actual Test Data')  
    plt.plot(test[model_name], color='green', linestyle='dashed', label=f'{model_name} Forecast')  
    plt.legend(loc='upper left')
    plt.title(f"{model_name} Forecasting")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.savefig(f"plots_naive/plot_{location}_{aux}.jpg", format='jpg', dpi=300)       
    

In [ ]:
def arima_model(train,test,mape,sp = 52):
    # finding best ARIMA and SARIMA model
    ARIMA_model = pm.auto_arima(
        train["value"],
        start_p=0, start_q=0,
        max_p=4, max_q=4,
        d=None,         
        max_d=2,
        seasonal=False,  
        stepwise=True,
        test='adf',      
        trace=False,
        suppress_warnings=True,
        error_action='ignore'
    )
    ARIMA_model.fit(train['value'])
    order_arima = ARIMA_model.get_params()['order']

    SARIMA_model = pm.auto_arima(
        train["value"],
        start_p=0, start_q=0,
        max_p=4, max_q=4,
        start_P=0, max_P=2,
        start_Q=0, max_Q=2,
        max_d=2, max_D=1,
        m=52,  # 52 weeks = 1 year seasonality
        d=None, D=1,
        seasonal=True,
        test='adf',
        trace=False,
        stepwise=True,
        suppress_warnings=True,
        error_action='ignore'
    )
    SARIMA_model.fit(train['value'])
    order_sarima = SARIMA_model.get_params()['order']
    
    # Forecasting



In [17]:
train, test = train_data(ari, "BE", "2023-10-13")
